In [2]:
import random
import json
import numpy as np
import torch
import math
from nltk.tokenize import word_tokenize, sent_tokenize
import csv
import pandas as pd
import random

from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser('http://localhost:9000', encoding="utf-8")


In [10]:
def clean_str(sent):
    sent = sent.replace("’", "'")
    sent = sent.replace("‘", "`")
    sent = sent.replace('"', "''")
    sent = sent.replace("—", "--")
    sent = sent.replace("…", "...")
    sent = sent.replace("–", "--")

    return sent

def get_dataset(scored_sentences_path, dataset_path, dataset_name, top_n, parser):

    if dataset_name == 'liar_plus':
        df = pd.read_csv(dataset_path, sep='\t', index_col=0)
        df = df.dropna()
        columns = ['dummy', 'id', 'statement', 'justification',
               'ruling_without_summary', 'label', 'just_tokenized',
               'ruling_tokenized', 'statement_tokenized', 'oracle_ids']
        print(df.columns)
        print(columns)
        df.columns = columns
        
    elif dataset_name == 'pub_health':
        df = pd.read_csv(dataset_path, sep='\t', index_col=0)
        df = df.dropna()
        
        columns = ['claim_id', 'claim', 'date_published', 'explanation',
                   'fact_checkers', 'main_text', 'sources', 'label', 'subjects']
        
        if len(df.columns) == 10:
            columns = ['dummy'] + columns
        
        df.columns = columns
        
    scored_sentences = [json.loads(line) for line in open(scored_sentences_path)]
    scored_sentences = {item["id"]: sorted(item['sentence_scores'], key=lambda x: x[1], reverse=True)[:top_n] for item in scored_sentences}
    
    
    inp_scored_sentences = {}
    for k, v in scored_sentences.items():
        
        temp = []
        for sent in v:
            temp.append(sent[0])
        inp_scored_sentences[k] = clean_str(" ".join(temp))

    scored_sentences = inp_scored_sentences
    
    
    if dataset_name == 'liar_plus':
        
        df['scored_sentences'] = df.apply(lambda x: scored_sentences.get(x['id'], None), axis=1)
        df = df[df['scored_sentences'] != None]
        df['justification_sentences'] = df.apply(lambda x: sent_tokenize(x['justification']), axis=1)
        df = df[['id', 'statement', 'justification', 'label', 'scored_sentences',
             'justification_sentences']]
        
    elif dataset_name == 'pub_health':
        df['claim_id'] = df['claim_id'].astype('str')
        df['scored_sentences'] = df.apply(lambda x: scored_sentences.get(x['claim_id'], None), axis=1)
        df = df[df['scored_sentences'] != None]
        df['justification_sentences'] = df.apply(lambda x: sent_tokenize(x['explanation']), axis=1)
        df = df[['claim_id', 'claim', 'explanation', 'label', 'scored_sentences',
             'justification_sentences']]
        
        
    dataset = [row.to_dict() for i, row in df.iterrows()]
    new_dataset = []
    if dataset_name == 'liar_plus':
        for i in dataset:
            if i["scored_sentences"] is None or i["id"] == '2001.json': #Sentence in Liarplus is too long:
                continue
            else:
                new_dataset.append(i)
    elif dataset_name == 'pub_health':
        for i in dataset:
        
            if i["scored_sentences"] is None or i["scored_sentences"] == None:
                continue
            else:
                new_dataset.append(i)
    

    print(f'Size of dataset: {len(dataset)}')
    print(f'Size of new dataset: {len(new_dataset)}')
    print('Sample: ', dataset[0])
    if len(new_dataset)!=0:
        print('Sample: ', new_dataset[0])

    return 

scored_sentences_path = "../../DATA-COPE-Project-DIKUServer/unsup_scores_liar/sentence_scores_val.jsonl" #Each line is a json
scored_sentences_path1 = "../../DATA-COPE-Project-DIKUServer/unsup_scores_pubhealth/sentence_scores_test.jsonl"

dataset_path = "../../liar_data/ruling_oracles_val.tsv"
dataset_path1 = "../../DATA-COPE-Project-DIKUServer/PUBHEALTH/test.tsv"

get_dataset(scored_sentences_path1, dataset_path1, 'pub_health', 6, parser)
#get_dataset(scored_sentences_path, dataset_path, 'liar_plus', 6, parser)


Size of dataset: 1007
Size of new dataset: 1007
Sample:  {'claim_id': '33456', 'claim': 'A mother revealed to her child in a letter after her death that she had just one eye because she had donated the other to him.', 'explanation': 'The one-eyed mother story expounds upon two moral messages: the unconditional, all-encompassing love we expect mothers to always feel for their children, and the admonition to not put off cherishing loved ones and appreciating their sacrifices while they’re still around.', 'label': 'false', 'scored_sentences': "By February 2007 the item was circulating in e-mail in the following shortened version:  My mom only had one eye. I confronted her that day and said, “If you're only gonna make me a laughing stock, why don't you just die?” My mom did not respond... I didn't even stop to think for a second about what I had said, because I was full of anger. In April 2005, we spotted a tearjerker on the Internet about a mother who gave up one of her eyes to a son who 